In [1]:
import pandas as pd
import numpy as np
import random as rd
import seaborn as sbs
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.svm import SVR
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow import data
import tensorflow as tf
from matplotlib import pyplot
import time
import csv
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error

In [2]:
file_data = 'DS_Agua_2017_2022_por_ponto.csv'
path_name='data/'
path_name_results='results/'
file_result = 'Result_SVR-LSTM_Model_Week.csv'

In [3]:
dataset = pd.read_csv(f'{path_name}{file_data}', sep =';', encoding = 'latin1', decimal='.')

In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 368849 entries, 1 to 368849
Data columns (total 8 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   DT_MEDICAO_HORA         368849 non-null  object 
 1   PRECIPITACAO            368849 non-null  float64
 2   PRESSAO_ATMOSFERICA     368849 non-null  float64
 3   TEMPERATURA_DO_AR_C     368849 non-null  float64
 4   UMIDADE_RELATIVA_DO_AR  368849 non-null  float64
 5   VELOCIDADE_VENTO        368849 non-null  float64
 6   SK_PONTO                368849 non-null  int64  
 7   VL_MEDICAO              368849 non-null  float64
dtypes: float64(6), int64(1), object(1)
memory usage: 25.3+ MB


In [5]:
#verifica se existe variáveis nulas 
dataset.isna().sum()

DT_MEDICAO_HORA           0
PRECIPITACAO              0
PRESSAO_ATMOSFERICA       0
TEMPERATURA_DO_AR_C       0
UMIDADE_RELATIVA_DO_AR    0
VELOCIDADE_VENTO          0
SK_PONTO                  0
VL_MEDICAO                0
dtype: int64

In [6]:
def salvar_resultado(sk_ponto, ds_best_param, n_time_steps, MSE, RMSE, MAE, MAPE, Duration):
  #Script to write training cycle results
  data = [sk_ponto, ds_best_param, n_time_steps, MSE, RMSE, MAE, MAPE, Duration]
  fields = ['SK_PONTO','Best Params','N_Past_Vl','MSE','RMSE','MAE','MAPE','Duration']
  with open(f'{path_name_results}{file_result}', "a",newline='') as csv_file:
    writer = csv.writer(csv_file,delimiter=';')
    writer.writerow(data)  
  print(fields)
  print(data)
    
#Script to create the results file
def criar_arquivo_resultado():
  fields = ['SK_PONTO','Best Params','N_Past_Vl','MSE','RMSE','MAE','MAPE','Duration']
  with open(f'{path_name_results}{file_result}', "w",newline='') as csv_file:
    writer = csv.writer(csv_file,delimiter=';')
    writer.writerow(fields)   

In [7]:
def imprime_grafico(y_test, predict):
  #O gráfico abaixo mostra os valores reais em preto e os valores prditos em azul
  npredicts=1000
  pyplot.figure(figsize=(10,7)) 
  pyplot.scatter(np.arange(0,y_test.shape[0],1)[0:npredicts],np.array(y_test)[0:npredicts], color = 'black')
  pyplot.plot(np.arange(0,predict.shape[0],1)[0:npredicts], np.array(predict)[0:npredicts], color = 'blue', linewidth=2)
  pyplot.xlabel('n seq.')
  pyplot.ylabel('consumo l/h')
  pyplot.show()

In [8]:
#crop date time to date 
dataset['DATA']=dataset['DT_MEDICAO_HORA'].str.split(' ').str[0]
dataset.drop(['DT_MEDICAO_HORA'],axis=1, inplace=True)

# Grouping data for Week

dataset['DATA'] = pd.to_datetime(dataset['DATA']) - pd.to_timedelta(7, unit='d')

dataset = dataset.groupby(['SK_PONTO',pd.Grouper(key='DATA', freq='W')], as_index=False).agg({'PRECIPITACAO':'sum','PRESSAO_ATMOSFERICA':'mean','TEMPERATURA_DO_AR_C':'mean','UMIDADE_RELATIVA_DO_AR':'mean','VELOCIDADE_VENTO':'mean','VL_MEDICAO':'sum'})
# cols = 'PRECIPITACAO':'sum','PRESSAO_ATMOSFERICA':'mean','TEMPERATURA_DO_AR_C':'mean','UMIDADE_RELATIVA_DO_AR':'mean','VELOCIDADE_VENTO':'mean','VL_MEDICAO':'sum'
#dataset = dataset.groupby(['SK_PONTO','DATA'], as_index=False).agg({'PRECIPITACAO':'sum','PRESSAO_ATMOSFERICA':'mean','TEMPERATURA_DO_AR_C':'mean','UMIDADE_RELATIVA_DO_AR':'mean','VELOCIDADE_VENTO':'mean','VL_MEDICAO':'sum'})
#df_result.rename(columns=cols)

In [9]:
def previsao_SVR(sk_ponto, dataset, n_time_steps):

    # dataframe tratament
  df = pd.DataFrame()
  df['VL_MEDICAO']=dataset['VL_MEDICAO']  

  # time serire transform - shit 1 step time

  for n_step in range(1,n_time_steps+1,1):
    df['vl-'+str(n_step)]=dataset['VL_MEDICAO'].shift(n_step)  
    
  df['tp-1']=dataset['TEMPERATURA_DO_AR_C']
  df['pr-1']=dataset['PRESSAO_ATMOSFERICA']
  df['vv-1']=dataset['VELOCIDADE_VENTO']
  df['ur-1']=dataset['UMIDADE_RELATIVA_DO_AR']
  df['ch-1']=dataset['PRECIPITACAO']  

  df.dropna(inplace=True)
  
  
  
  #Split dataset in treinam /  75% treinam  25% test
  nlinhas = int(np.round(df.shape[0] *0.75)) # 
  
  max_size_train_split = int(np.round(nlinhas / 5)) 
  max_size__test_split = int(np.round((df.shape[0] - nlinhas) / 5))
  size_split = 5
  X_train = df.iloc[0:nlinhas,1: 6 + n_time_steps]
  y_train = df.iloc[0:nlinhas,0].values

  X_test = df.iloc[nlinhas:dataset.shape[0],1: 6 + n_time_steps] 
  y_test = df.iloc[nlinhas:dataset.shape[0],0].values
  
  
  # Stores the training execution start time
  Hora_Inicio = time.time()
  
  #Create SVR Regressor
  modelo  = SVR(kernel='rbf')
  
  # Validados cruzado para séries temporais para 5 conjuntos de dados
  ts_cv = TimeSeriesSplit(
      n_splits=size_split, # Number of divisions
      max_train_size=max_size_train_split,    # maximum size of each set. of training
      gap=2, # number of samples to exclude between each training and testing set
      test_size=max_size__test_split, # maximum size of each set. of test.
  )
  C = [ 12550, 125550, 1255555] # Parâmetro de regularização
  gamma = [ 0.00001, 0.000001, 0.0000001, 0.00000001] # Coeficiente da função kernel  
  epsilon = [0.1, 0.01, 0.001, 0.0001]

  hyper_params = [{'kernel': ['rbf'],'C': C, 'gamma':gamma, 'epsilon':epsilon}]  

  modelo = SVR(max_iter=500)
  grid = GridSearchCV(modelo,param_grid=hyper_params,verbose=20,n_jobs=-4,cv=ts_cv, scoring='neg_mean_absolute_percentage_error')
  grid.fit(np.array(X_train), np.array(y_train))
  
  resultado = str(grid.best_params_)
  

  predict=grid.predict(np.array(X_test))   
  
  #Estimate Et for treinam and test
  # For treinam data
  Lt_train = grid.predict(X_train)
  Yt_train = y_train
  Et_train = Yt_train - Lt_train  

  # For test data
  Lt_test = predict
  Yt_test = y_test
  Et_test = Yt_test - Lt_test  

  #create  dataframe of Et value
  df_Et = pd.DataFrame(Et_train, columns=['VL_Et'])  

  for n_step in range(1,n_time_steps+1,1):
      df_Et['vl-'+str(n_step)]=df_Et['VL_Et'].shift(n_step)    

  #drop rows with null values
  df_Et.dropna(inplace=True)  

  Nt_train = df_Et['VL_Et']  

  df_Et.drop(['VL_Et'],axis=1, inplace=True)  

  X_train = df_Et
  y_train = Nt_train
  
  #LSTM Model for hybrid prediction for non linear component Et_train 
  
  train_feature = np.array(X_train).reshape((X_train.shape[0], 1, X_train.shape[1])) # dados de entrada de treinamento.
  train_label = np.array(y_train).reshape((y_train.shape[0], 1, 1)) # dado de saída esperados no treinamento.  

  N_Nos_Layer1=100  #32 #Número de neurônios do layer 1   # 64
  N_Nos_Layer2=96   #64 # Número de neurônios do layer 2  # 128
  N_Nos_Layer3=90   #72 # Númer ode neurônios do layer 3  
  
  nlinhas = X_train.shape[0]
  batch_size = 4096
  if(batch_size > nlinhas):
    batch_size = nlinhas
    
  train_data = data.Dataset.from_tensor_slices((train_feature, train_label))
  train_data = train_data.repeat().batch(batch_size, drop_remainder=True)
  EPOCAS = 200  

  model = keras.Sequential()
  model.add(layers.LSTM(N_Nos_Layer1, batch_input_shape=(batch_size, train_feature.shape[1], train_feature.shape[2]),  return_sequences=True, activation='relu'))  
  model.add(layers.LSTM(N_Nos_Layer2, batch_input_shape=(N_Nos_Layer1, train_feature.shape[1], train_feature.shape[2]), return_sequences=True, activation='relu'))  
  model.add(layers.Dense(N_Nos_Layer3, activation="relu"))
  model.add(layers.Dense(1))

  model.compile(loss='mean_absolute_percentage_error', optimizer='adam', metrics=['mape'])  

  resultado = 'SVR-LSTM' + resultado +'-' + 'LSTM('+ str(N_Nos_Layer1) +','+str(N_Nos_Layer2)+','+str(N_Nos_Layer3)+')'

  #history = model.fit(train_feature, train_label, epochs=EPOCAS,shuffle=True,batch_size=batch_size, verbose=2)
  steps_per_epoch = len(train_feature) // batch_size 
  history = model.fit(train_data, epochs=EPOCAS, steps_per_epoch = steps_per_epoch) 
  
  #--------------------------------------------
  
  #create  dataframe of Et value
  df_Et = pd.DataFrame(Et_test, columns=['VL_Et'])
  
  for n_step in range(1,n_time_steps+1,1):
      df_Et['vl-'+str(n_step)]=df_Et['VL_Et'].shift(n_step)  
  
  #drop rows with null values
  df_Et.dropna(inplace=True)
  
  Nt_test = df_Et['VL_Et']
  
  df_Et.drop(['VL_Et'],axis=1, inplace=True)
  
  X_test = df_Et
  y_test = Nt_test
  
  ## predict test values for LSTM Model
  
  val_feature = np.array(X_test).reshape((X_test.shape[0], 1, X_test.shape[1]))
  val_label = np.array(y_test).reshape((y_test.shape[0], 1, 1))
  
  if(batch_size > X_test.shape[0] ):
    batch_size = X_test.shape[0]
  batch_size=1
  test_data = data.Dataset.from_tensor_slices((val_feature, val_label))
  test_data = test_data.repeat().batch(batch_size, drop_remainder=True)
  
  predict = model.predict(test_data, steps=1)
  #predict = model.predict(train_data, steps=1)
  
  val_label=val_label[0:len(predict)]
  
  if(val_label.shape[0] < predict.shape[0]):
    predict=predict[0:val_label.shape[0]]
  else:
    val_label=val_label[0:predict.shape[0]]
  
  if(batch_size > predict.shape[0]):
    batch_size = predict.shape[0]
    
  predict = np.array(predict).reshape(batch_size,1)
  y_test = np.array(val_label).reshape(batch_size,1)
  
  # Recomposition of Nt + Lt form test data
  Lt_test2 = Lt_test[n_time_steps:len(Lt_test)]
  Yt_test2 = Yt_test[n_time_steps:len(Yt_test)]
  Yt_test_predicted = predict + Lt_test2
  MAPE = ((np.mean(np.abs(Yt_test2 - Yt_test_predicted) / (Yt_test2)))) * 100
  
  print('MAPE',MAPE)
  
  predict = Yt_test_predicted[0]
  y_test = Yt_test2
  print('predict=',predict.shape) 
  print('y_test=',y_test.shape) 
  
  # Stores the training execution end time
  Hora_Fim = time.time()   

  #Calculate the duration of the training execution
  Duracao = Hora_Fim - Hora_Inicio   

  #Mean Squared Error (Mean Squared Difference Between Estimated Values and Actual Values) - MSE
  MSE = mean_squared_error(y_test, predict)   

  # Square Root of Mean Error - RMSE
  RMSE = np.sqrt(mean_squared_error(y_test, predict))   

  # Mean Absolute Distance or Mean Absolute Error - MAE
  MAE= median_absolute_error(y_pred=predict, y_true = y_test)   

  #Calculate the MAPE (Mean Absolute Percentage Error)
  MAPE = ((np.mean(np.abs(y_test -predict) / (y_test)))) * 100    

  #O gráfico abaixo mostra os valores reais em preto e os valores prditos em azul
  #imprime_grafico(Yt_train2, Yt_train_predicted)

  salvar_resultado(sk_ponto, resultado, n_time_steps, MSE, RMSE, MAE, MAPE, Duracao)

In [10]:
#selects the identification points of each reservoir
df_unique_pontos = dataset['SK_PONTO'].unique()

#create file to results
criar_arquivo_resultado()

for sk in np.array(df_unique_pontos):
    print('forecast for sk_ponto = ',sk)
    df_ponto = dataset[dataset['SK_PONTO']==sk]
    df_ponto = df_ponto.drop('SK_PONTO',axis=1)
    for n_time_steps in range(1,7): #predict with 1 to 6 past values of medition
        previsao_SVR(sk, df_ponto, n_time_steps)

forecast for sk_ponto =  1
Fitting 5 folds for each of 48 candidates, totalling 240 fits


c:\Python\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but SVR was fitted without feature names
  warnings.warn(


Epoch 1/200
1/1 [==============================] - 4s 4s/step - loss: 10757.7578 - mape: 10757.7578
Epoch 2/200
1/1 [==============================] - 0s 8ms/step - loss: 6917.1953 - mape: 6917.1953
Epoch 3/200
1/1 [==============================] - 0s 16ms/step - loss: 3194.2690 - mape: 3194.2690
Epoch 4/200
1/1 [==============================] - 0s 32ms/step - loss: 6654.7075 - mape: 6654.7075
Epoch 5/200
1/1 [==============================] - 0s 16ms/step - loss: 6855.4512 - mape: 6855.4512
Epoch 6/200
1/1 [==============================] - 0s 16ms/step - loss: 6146.3491 - mape: 6146.3491
Epoch 7/200
1/1 [==============================] - 0s 16ms/step - loss: 5566.9341 - mape: 5566.9341
Epoch 8/200
1/1 [==============================] - 0s 16ms/step - loss: 4321.6870 - mape: 4321.6870
Epoch 9/200
1/1 [==============================] - 0s 16ms/step - loss: 1884.0378 - mape: 1884.0378
Epoch 10/200
1/1 [==============================] - 0s 8ms/step - loss: 158.4057 - mape: 158.4057
Epo

c:\Python\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but SVR was fitted without feature names
  warnings.warn(


Epoch 1/200
1/1 [==============================] - 5s 5s/step - loss: 12499.6035 - mape: 12499.6035
Epoch 2/200
1/1 [==============================] - 0s 8ms/step - loss: 7328.8057 - mape: 7328.8057
Epoch 3/200
1/1 [==============================] - 0s 8ms/step - loss: 5031.1958 - mape: 5031.1958
Epoch 4/200
1/1 [==============================] - 0s 8ms/step - loss: 5190.8208 - mape: 5190.8208
Epoch 5/200
1/1 [==============================] - 0s 8ms/step - loss: 3953.1233 - mape: 3953.1233
Epoch 6/200
1/1 [==============================] - 0s 8ms/step - loss: 2454.5171 - mape: 2454.5171
Epoch 7/200
1/1 [==============================] - 0s 8ms/step - loss: 1399.7952 - mape: 1399.7952
Epoch 8/200
1/1 [==============================] - 0s 8ms/step - loss: 1571.2383 - mape: 1571.2383
Epoch 9/200
1/1 [==============================] - 0s 0s/step - loss: 1528.5107 - mape: 1528.5107
Epoch 10/200
1/1 [==============================] - 0s 8ms/step - loss: 1306.5082 - mape: 1306.5082
Epoch 11/

...

Epoch 1/200
1/1 [==============================] - 6s 6s/step - loss: 1161467.3750 - mape: 1161467.3750
Epoch 2/200
1/1 [==============================] - 0s 12ms/step - loss: 600836.9375 - mape: 600836.9375
Epoch 3/200
1/1 [==============================] - 0s 17ms/step - loss: 97416.8594 - mape: 97416.8594
Epoch 4/200
1/1 [==============================] - 0s 24ms/step - loss: 550813.5625 - mape: 550813.5625
Epoch 5/200
1/1 [==============================] - 0s 32ms/step - loss: 814792.4375 - mape: 814792.4375
Epoch 6/200
1/1 [==============================] - 0s 18ms/step - loss: 826473.8125 - mape: 826473.8125
Epoch 7/200
1/1 [==============================] - 0s 16ms/step - loss: 742786.8125 - mape: 742786.8125
Epoch 8/200
1/1 [==============================] - 0s 17ms/step - loss: 580672.3750 - mape: 580672.3750
Epoch 9/200
1/1 [==============================] - 0s 41ms/step - loss: 371703.8750 - mape: 371703.8750
Epoch 10/200
1/1 [==============================] - 0s 4ms/step - 

c:\Python\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Python\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but SVR was fitted without feature names
  warnings.warn(


Epoch 1/200
1/1 [==============================] - 3s 3s/step - loss: 1300439.6250 - mape: 1300439.6250
Epoch 2/200
1/1 [==============================] - 0s 10ms/step - loss: 764579.8750 - mape: 764579.8750
Epoch 3/200
1/1 [==============================] - 0s 16ms/step - loss: 159295.0625 - mape: 159295.0625
Epoch 4/200
1/1 [==============================] - 0s 13ms/step - loss: 348681.0000 - mape: 348681.0000
Epoch 5/200
1/1 [==============================] - 0s 9ms/step - loss: 345655.0312 - mape: 345655.0312
Epoch 6/200
1/1 [==============================] - 0s 8ms/step - loss: 352248.0000 - mape: 352248.0000
Epoch 7/200
1/1 [==============================] - 0s 8ms/step - loss: 252152.2500 - mape: 252152.2500
Epoch 8/200
1/1 [==============================] - 0s 8ms/step - loss: 146680.9375 - mape: 146680.9375
Epoch 9/200
1/1 [==============================] - 0s 8ms/step - loss: 80276.7656 - mape: 80276.7656
Epoch 10/200
1/1 [==============================] - 0s 8ms/step - loss:

c:\Python\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Python\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but SVR was fitted without feature names
  warnings.warn(


Epoch 1/200
1/1 [==============================] - 3s 3s/step - loss: 512491.6562 - mape: 512491.6562
Epoch 2/200
1/1 [==============================] - 0s 0s/step - loss: 402673.0312 - mape: 402673.0312
Epoch 3/200
1/1 [==============================] - 0s 8ms/step - loss: 299263.7188 - mape: 299263.7188
Epoch 4/200
1/1 [==============================] - 0s 16ms/step - loss: 220077.5469 - mape: 220077.5469
Epoch 5/200
1/1 [==============================] - 0s 10ms/step - loss: 126136.4531 - mape: 126136.4531
Epoch 6/200
1/1 [==============================] - 0s 24ms/step - loss: 55785.8359 - mape: 55785.8359
Epoch 7/200
1/1 [==============================] - 0s 8ms/step - loss: 16544.3672 - mape: 16544.3672
Epoch 8/200
1/1 [==============================] - 0s 9ms/step - loss: 47155.9922 - mape: 47155.9922
Epoch 9/200
1/1 [==============================] - 0s 8ms/step - loss: 62568.4180 - mape: 62568.4180
Epoch 10/200
1/1 [==============================] - 0s 16ms/step - loss: 66264.1